In [4]:
import os

# Handle to the workspace
from azure.ai.ml import MLClient

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command, Input, Output
from azure.ai.ml import dsl

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, AzureCliCredential

# Maybe we need to access the storage account
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient

try:
    credential = AzureCliCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
ml_client = MLClient.from_config(credential)


AzureCliCredential.get_token failed: ERROR: AADSTS70043: The refresh token has expired or is invalid due to sign-in frequency checks by conditional access. The token was issued on 2025-04-01T06:22:55.4629892Z and the maximum allowed lifetime for this request is 14400. Trace ID: 06009eb9-ecac-4f0d-ad3a-eae61dcb2300 Correlation ID: 2c8c1d48-f167-4736-80c8-80dc9bb7ca19 Timestamp: 2025-04-03 10:41:39Z
Interactive authentication is needed. Please run:
az login --scope https://management.azure.com/.default

Found the config file in: /home/daniel/repos/aml_demo/config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [2]:
os.mkdir("../components/readfromdatastore")

In [5]:
%%writefile ../components/readfromdatastore/readfromdatastore.py

import os
import argparse

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--input_path", type=str, help="path to datastore")

    #list files in the path
    args = parser.parse_args()
    path = args.input_path
    print(f"Files in {path}:")
    for file in os.listdir(path):
        print(file)
    print(f"Read from {path} successfully.")
    print("Read from datastore successfully.")
if __name__ == "__main__":
    main()


Overwriting ../components/readfromdatastore/readfromdatastore.py


In [49]:
inputs = {"input_path": Input(type="uri_folder", mode = "ro_mount")}

In [50]:
read_command = command(
    name="readfromdatastore",
    display_name="readfromdatastore",
    description="Read from datastore",
    inputs=inputs,
    outputs={},
    code="../components/readfromdatastore",
    command="python readfromdatastore.py --input_path ${{inputs.input_path}}",
    environment="yolofromdocker:1",
    compute="defaultcompute",
    experiment_name="readfromdatastore",
)

In [51]:
@dsl.pipeline(
    name="readfromdatastore",
    description="Read from datastore",
)
def readfromdatastore(input_path: str): 
    read_command_job = read_command(input_path=input_path)
    return read_command_job

In [60]:
ml_client.jobs.create_or_update(readfromdatastore(input_path=Input(type="uri_folder", mode = "ro_mount", path="azureml://datastores/container_raw/paths/coco128")))

Experiment,Name,Type,Status,Details Page
notebooks,wheat_yuca_3sqpn8v0lw,pipeline,NotStarted,Link to Azure Machine Learning studio


In [ ]:
ml_client.datastores.get("container_raw")

AttributeError: 'AzureBlobDatastore' object has no attribute 'list_paths'